In [1]:
import os, re
import pandas as pd
import glob

In [2]:
import re
import pandas as pd

def extract_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        log_content = file.read()

    # Split log by task descriptions
    task_sections = re.split(r'(Task Description:.*?)(?=\n\d{4}-\d{2}-\d{2})', log_content, flags=re.DOTALL)

    data = []
    for i in range(1, len(task_sections), 2):
        task_description = task_sections[i].strip()
        if task_description.startswith('Task Description:\n'):
            task_description = task_description[len('Task Description:\n'):].strip()
        steps_content = task_sections[i + 1].strip()

      
        steps = re.findall(r'Output by GPT: (.*?)\n.*?Action given by GPT: (.*?)\n.*?Step: (\d+).*?Observation:\s*>>>([\s\S]*?)(?=\n\d{4}-\d{2}-\d{2}|\Z)\n.*?Reward: (-?\d+)\n.*?Score: (-?\d+)', steps_content, re.DOTALL)

        for step in steps:
            step_num = int(step[2])
            thought_action = step[0].split('Action:', 1)
            thought = thought_action[0].replace('Thought:', '').strip()
            action = re.split(r'\n', step[1].strip())[0] 

            full_observation = step[3].strip()
          
            reward=int(step[4])
            score=int(step[5])
            

            
            data.append({
                'task_description': task_description,
                'step': step_num,
                'thought': thought,
                'action': action,
                'observation': full_observation,
                'reward': reward,
                'score': score
            })
    
    df = pd.DataFrame(data)
    df = df[df['action'].str.strip() != 'task'].reset_index(drop=True)
    return df[['task_description', 'step', 'thought', 'action', 'observation', 'reward', 'score']]

In [3]:
def process_files_in_folder(folder_path):
    combined_df = pd.DataFrame()
    
    # Get list of all files in the folder
    file_paths = glob.glob(os.path.join(folder_path, '*.log'))
    file_paths.remove(os.path.join(folder_path, 'results_data_10.log'))
 
    
    for file_path in file_paths:
        df = extract_data(file_path)
        combined_df = pd.concat([combined_df, df], ignore_index=True)
    
    return combined_df

In [4]:
folder_path = 'logs_sc/react/train_data' 
combined_df = process_files_in_folder(folder_path)

In [5]:
combined_df

,task_description,step,thought,action,observation,reward,score
0,Your task is to measure the melting point of m...,1,I need to go to the kitchen to find the blue b...,go to kitchen,You move to the kitchen.,6,6
1,Your task is to measure the melting point of m...,2,I need to look for the blue box in the kitchen.,look for blue box,No known action matches that input.,0,6
2,Your task is to measure the melting point of m...,3,I need to find the blue box in the kitchen.,look at blue box,a blue box (containing nothing),0,6
3,Your task is to measure the melting point of m...,4,"I should focus on the blue box, as instructed,...",focus on blue box,You focus on the blue box.,-106,-100
4,Your task is to measure the melting point of m...,1,"I am currently in the art studio, not the kitc...",go to kitchen,No known action matches that input.,0,0
...,...,...,...,...,...,...,...
24184,Your task is to turn on the green light bulb b...,14,I need to examine the nuclear generator and th...,examine nuclear generator,"a nuclear generator, which is off. its anode i...",0,10
24185,Your task is to turn on the green light bulb b...,15,I should connect the green wire to the nuclear...,connect green wire terminal 1 to anode in nucl...,No known action matches that input.,0,10
24186,Your task is to turn on the green light bulb b...,16,I need to connect the green wire to the nuclea...,connect green wire terminal 1 to anode in nucl...,terminal 1 on green wire is now connected to a...,0,10
24187,Your task is to turn on the green light bulb b...,17,Now that one end of the green wire is connecte...,connect green wire terminal 2 to green light bulb,Connections must specify specific points of co...,0,10


In [6]:
combined_df.iloc[2]['observation']

'a blue box (containing nothing)'

In [7]:
def save_dataframe(df, folder_name='pretraining_data', file_name='data.csv'):
    # Ensure the directory exists
    os.makedirs(folder_name, exist_ok=True)
    # Save the DataFrame as a CSV file
    file_path = os.path.join(folder_name, file_name)
    df.to_csv(file_path, index=False)
    print(f'DataFrame saved to {file_path}')

In [8]:
save_dataframe(combined_df)

DataFrame saved to pretraining_data/data.csv


In [9]:
mini_100=combined_df.head(100)

In [10]:
save_dataframe(mini_100, file_name='mini_data.csv')

DataFrame saved to pretraining_data/mini_data.csv
